In [7]:
# importing libraries, etc...

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

The [Gapminder](https://www.gapminder.org/) dataset contains historical data (mid-19th century onwards) containing hundreds of indicators such as life expectancy and GDP for countries around the world.
For our purpose, we will try to predict the life expectancy of countries based on several of these indicators.

To make experimenting with Cross-Validation and Grid Search on the life_expectancy dataset a bit more feasible, I have only included data from the year 2018. I have titled this subset of the life_expectancy dataset life_expectancy.csv.

In [8]:
life_expectancy = pd.read_csv("life_expectancy.csv")

life_expectancy

,geo,child_mortality_0_5_year_olds_dying_per_1000_born,children_and_elderly_per_100_adults,children_per_woman_total_fertility,children_per_woman_total_fertility_with_projections,crude_death_rate_deaths_per_1000_population,income_per_person_gdppercapita_ppp_inflation_adjusted,population_growth_annual_percent_with_projections,population_total,life_expectancy_sex_ratio,population_sex_ratio,life_expectancy_years
0,afg,65.924,80.353,4.33,3.941,6.443,1867.0,2.309,36373176.0,0.958807,1.062219,58.69
1,ago,81.638,98.221,5.55,5.367,8.243,5846.0,3.255,30774205.0,0.911825,0.962679,65.19
2,arg,10.620,56.313,2.26,2.113,7.557,18942.0,0.930,44688864.0,0.907896,0.958863,76.97
3,arm,12.920,44.200,1.60,1.744,9.688,8662.0,0.107,2934152.0,0.918373,0.888299,75.97
4,aus,3.398,53.564,1.83,1.872,6.695,45783.0,1.288,24772247.0,0.956382,0.992728,82.87
...,...,...,...,...,...,...,...,...,...,...,...,...
194,grd,14.630,50.986,2.06,2.062,7.094,13505.0,0.462,108339.0,0.935144,1.009945,71.86
195,kir,52.532,64.731,3.57,2.801,6.944,1888.0,1.702,118414.0,0.905009,0.972727,62.23
196,syc,12.674,45.796,2.25,2.074,8.330,27546.0,0.507,95235.0,0.885215,0.971965,74.23
197,fsm,32.766,60.715,3.05,3.055,6.235,3409.0,0.677,106227.0,0.965438,1.052121,65.80


In [9]:
# As usual, we scale the features

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler().fit(life_expectancy.iloc[:, 1:-1])
life_expectancy.iloc[:, 1:-1] = scaler.transform(life_expectancy.iloc[:, 1:-1])

In [10]:
# checking the number of missing values per feature

life_expectancy.isna().sum()

geo                                                       0
child_mortality_0_5_year_olds_dying_per_1000_born         1
children_and_elderly_per_100_adults                       0
children_per_woman_total_fertility                        0
children_per_woman_total_fertility_with_projections       0
crude_death_rate_deaths_per_1000_population               0
income_per_person_gdppercapita_ppp_inflation_adjusted    11
population_growth_annual_percent_with_projections         0
population_total                                          0
life_expectancy_sex_ratio                                 0
population_sex_ratio                                      0
life_expectancy_years                                     0
dtype: int64

In [11]:
# Imputing missing values using the k-NN algorithm, with n_neighbors=3

from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=3).fit(life_expectancy.iloc[:,1:])
life_expectancy.iloc[:,1:] = imputer.transform(life_expectancy.iloc[:,1:])

In [12]:
life_expectancy.isna().sum()

geo                                                      0
child_mortality_0_5_year_olds_dying_per_1000_born        0
children_and_elderly_per_100_adults                      0
children_per_woman_total_fertility                       0
children_per_woman_total_fertility_with_projections      0
crude_death_rate_deaths_per_1000_population              0
income_per_person_gdppercapita_ppp_inflation_adjusted    0
population_growth_annual_percent_with_projections        0
population_total                                         0
life_expectancy_sex_ratio                                0
population_sex_ratio                                     0
life_expectancy_years                                    0
dtype: int64

In [14]:
# Splitting into training and test set

le_features_train, le_features_test, le_target_train, le_target_test = train_test_split(life_expectancy.iloc[:,1:-1],
                                                                                       life_expectancy.iloc[:,-1],
                                                                                       test_size=0.35,
                                                                                       random_state=99)

In [ ]:
# Ignore this, just some code to allow the display of wide plots
from IPython.display import display, HTML
CSS = """div.output_area img {max-width:None !important;max-height: None !important";}"""
display(HTML('<style>{}</style>'.format(CSS)))

In [ ]:
sns.pairplot(data=life_expectancy,
            x_vars=life_expectancy.columns[1:-1],
             y_vars=life_expectancy.columns[-1], height=4)

In [ ]:
life_expectancy.drop(columns="children_per_woman_total_fertility_with_projections", inplace=True)

In [ ]:
# using Grid Search and Cross Validation to find the optimal parameters. Here, I have used 10 folds, but feel free to use 
# more or fewer in the model(s) you make below!

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor

params = {
    "n_neighbors": [1, 3, 5, 7, 9, 11],
    "weights": ["uniform", "distance"]
}

knn = GridSearchCV(estimator=KNeighborsRegressor(),
                   param_grid=params, cv=10) 

knn.fit(le_features_train, le_target_train)

print("Training set score: {:.4f}".format(knn.score(le_features_train, le_target_train)))
print("Test set score: {:.4f}".format(knn.score(le_features_test, le_target_test)))
print(knn.best_params_)

Now, it's your turn to use any of the models we've discussed to see how well they perform on this task. Since this dataset is significantly smaller than the mnist (handwritten digits) dataset, it is very feasible - and, practically a requirement - to use Grid Search and Cross Validation to build and test your models. Note that this is a regression problem and classification models will thus not work on it. Perhaps even more important than choosing a classifier is trying out different parameter settings (e.g. n_neighbors for k-Nearest Neighbors, C for Logistic Regression, n_estimators for the Random Forest Classifier, etc...). 

Below are the regression models we've discussed, along with the import statement and the parameters that we've covered during the sessions.

- **k-Nearest Neighbors Regression** (already imported in the cell above)
    - n_neighbors (any number above 0)
    - weights ("uniform", "distance")
- **Linear Regression** (from sklearn.linear_model import LinearRegression)
    - C
- **Ridge Regression** (from sklearn.linear_model import Ridge)
    - alpha (any number above 0)
- **Lasso Regression** (from sklearn.linear_model import Lasso)
    - alpha (any number above 0)
- **Decision Tree Regression** (from sklearn.tree import DecisionTreeRegressor)
    - max_depth (a whole number above 0)
    - min_samples_split (a whole number above 1)
- **Random Forest Regression** (from sklearn.ensemble import RandomForestRegressor)
    - n_estimators (a whole number above 0)
    - max_depth (a whole number above 0)
    - min_samples_split (a whole number above 1)
- **Gradient Boosting Regressor** (from sklearn.ensemble import GradientBoostingRegressor)
    - n_estimators (a whole number above 0)
    - max_depth (a whole number above 0)
    - min_samples_split (a whole number above 1)
    - learning_rate (a number between 0 and 1)
    - subsample (a number between 0 and 1)
    
If you want to access even more parameter settings than we've discussed in class (models tend to have a lot), you can also access the sklearn documentation. For example, [here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html), you can find all possible parameters to tune for the KNeighborsClassifier.

Good luck and feel free to share your model(s) (and the results you obtain with it) on the Canvas discussion page!

In [15]:
# First attempt: Linear Regression

from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(le_features_train, le_target_train)

print("Training set score: {:.4f}".format(lr.score(le_features_train, le_target_train)))
print("Test set score: {:.4f}".format(lr.score(le_features_test, le_target_test)))

Training set score: 0.8499
Test set score: 0.8034


In [ ]:
# Let's use a more complex model: Random Forest Regression, with parameters optimized using Grid Search + Cross-Validation

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

params = {
    "n_estimators": [500],
    "max_depth": [3, 6, None],
    "max_features": ["auto", 0.5, "sqrt"]
}

rfr = GridSearchCV(estimator=RandomForestRegressor(),
                   param_grid=params, cv=5, n_jobs=-1, verbose=1) 

rfr.fit(le_features_train, le_target_train)

print("Training set score: {:.4f}".format(rfr.best_score_))
print("Test set score: {:.4f}".format(rfr.score(le_features_test, le_target_test)))
print(rfr.best_params_)

In [ ]:
# Now let's go one step further and try Gradient Boosting, with parameters optimized using Grid Search + Cross-Validation
from sklearn.ensemble import GradientBoostingRegressor

params = {
    "n_estimators": [500],
    "learning_rate": [0.01, 0.05, 0.1, 0.15],
    "max_depth": [3, 6, None],
}

gbr = GridSearchCV(estimator=GradientBoostingRegressor(),
                   param_grid=params, cv=5, verbose=1) 

gbr.fit(le_features_train, le_target_train)

print(gbr.best_params_)
print("Training set score: {:.4f}".format(gbr.best_score_))
print("Test set score: {:.4f}".format(gbr.score(le_features_test, le_target_test)))

In [16]:
# To do even better, we might have to get a little more creative. 
# Let's go back to using linear models, but after adding polynomial features and interaction terms.

le_poly = pd.DataFrame([life_expectancy.geo]).T

for col in life_expectancy.iloc[:,1:-1].columns:
    le_poly[col] = life_expectancy[col]
    le_poly[f"{col}**2"] = life_expectancy[col]**2
    for col2 in life_expectancy.iloc[:,1:-1].columns:
        if col != col2:
            le_poly[f"{col} * {col2}"] = life_expectancy[col] * life_expectancy[col2]

le_poly["life_expectancy_years"] = life_expectancy["life_expectancy_years"]

C:\Users\lenna\AppData\Local\Temp/ipykernel_11764/199574975.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  le_poly[col] = life_expectancy[col]
C:\Users\lenna\AppData\Local\Temp/ipykernel_11764/199574975.py:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  le_poly[f"{col}**2"] = life_expectancy[col]**2
C:\Users\lenna\AppData\Local\Temp/ipykernel_11764/199574975.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider jo

In [17]:
le_poly

,geo,child_mortality_0_5_year_olds_dying_per_1000_born,child_mortality_0_5_year_olds_dying_per_1000_born**2,child_mortality_0_5_year_olds_dying_per_1000_born * children_and_elderly_per_100_adults,child_mortality_0_5_year_olds_dying_per_1000_born * children_per_woman_total_fertility,child_mortality_0_5_year_olds_dying_per_1000_born * children_per_woman_total_fertility_with_projections,child_mortality_0_5_year_olds_dying_per_1000_born * crude_death_rate_deaths_per_1000_population,child_mortality_0_5_year_olds_dying_per_1000_born * income_per_person_gdppercapita_ppp_inflation_adjusted,child_mortality_0_5_year_olds_dying_per_1000_born * population_growth_annual_percent_with_projections,child_mortality_0_5_year_olds_dying_per_1000_born * population_total,...,population_sex_ratio * child_mortality_0_5_year_olds_dying_per_1000_born,population_sex_ratio * children_and_elderly_per_100_adults,population_sex_ratio * children_per_woman_total_fertility,population_sex_ratio * children_per_woman_total_fertility_with_projections,population_sex_ratio * crude_death_rate_deaths_per_1000_population,population_sex_ratio * income_per_person_gdppercapita_ppp_inflation_adjusted,population_sex_ratio * population_growth_annual_percent_with_projections,population_sex_ratio * population_total,population_sex_ratio * life_expectancy_sex_ratio,life_expectancy_years
0,afg,0.514189,0.264391,0.334031,0.270167,0.227625,0.181947,0.005287,0.301371,1.318331e-02,...,0.054628,0.069018,0.055822,0.047032,0.037594,0.001092,0.062269,2.723933e-03,0.081582,58.69
1,ago,0.640482,0.410217,0.536494,0.468963,0.430986,0.310817,0.027752,0.482745,1.388694e-02,...,0.038433,0.050264,0.043937,0.040379,0.029121,0.002600,0.045228,1.301069e-03,0.025127,65.19
2,arg,0.069713,0.004860,0.027653,0.012170,0.010287,0.030339,0.010603,0.023826,2.197067e-03,...,0.004060,0.023100,0.010166,0.008593,0.025343,0.008857,0.019903,1.835307e-03,0.022685,76.97
3,arm,0.088198,0.007779,0.023744,0.005531,0.007760,0.052107,0.005884,0.017283,1.769574e-04,...,0.002245,0.006854,0.001597,0.002240,0.015040,0.001698,0.004989,5.107774e-05,0.011899,75.97
4,aus,0.011670,0.000136,0.004291,0.001187,0.001268,0.004344,0.004376,0.004729,2.035215e-04,...,0.000863,0.027200,0.007522,0.008036,0.027533,0.027738,0.029971,1.289951e-03,0.055464,82.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,grd,0.101941,0.010392,0.034723,0.014341,0.014203,0.040918,0.010902,0.026388,9.440851e-07,...,0.008355,0.027917,0.011530,0.011419,0.032898,0.008765,0.021216,7.590483e-07,0.048524,71.86
195,kir,0.406558,0.165290,0.197281,0.161245,0.105151,0.158734,0.004251,0.194563,6.660028e-06,...,0.026294,0.031383,0.025650,0.016727,0.025251,0.000676,0.030950,1.059454e-06,0.023806,62.23
196,syc,0.086221,0.007434,0.024660,0.014906,0.012180,0.042389,0.019275,0.023006,0.000000e+00,...,0.005546,0.018396,0.011120,0.009086,0.031622,0.014379,0.017163,0.000000e+00,0.014214,74.23
197,fsm,0.247699,0.061355,0.109728,0.076409,0.074222,0.083887,0.005719,0.073555,1.924245e-06,...,0.025154,0.044986,0.031326,0.030429,0.034392,0.002345,0.030156,7.888996e-07,0.082985,65.80


In [ ]:
le_poly

In [18]:
le_poly_features_train, le_poly_features_test, le_poly_target_train, le_poly_target_test = train_test_split(le_poly.iloc[:,1:-1],
                                                                                       le_poly.iloc[:,-1],
                                                                                       test_size=0.35,
                                                                                       random_state=99)

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression().fit(le_poly_features_train, le_target_train)

print("{:.4f}".format(lr.score(le_poly_features_test, le_target_test)))

In [19]:
from sklearn.linear_model import Lasso

lasso = Lasso(alpha=0.001, tol=0.001).fit(le_poly_features_train, le_poly_target_train)

print("{:.4f}".format(lasso.score(le_poly_features_test, le_poly_target_test)))

0.8764


C:\Users\lenna\Anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.897e+01, tolerance: 6.622e+00
  model = cd_fast.enet_coordinate_descent(


In [21]:
params = {
    "tol": [0.001],
    "alpha": [0.0005, 0.001, 0.005, 0.01, 0.05, 0.1]
}

lasso = GridSearchCV(estimator=Lasso(),
                   param_grid=params, cv=30, n_jobs=-1, verbose=1) 

lasso.fit(le_poly_features_train, le_poly_target_train)

print("{:.4f}".format(lasso.best_score_))
print("{:.4f}".format(lasso.score(le_poly_features_test, le_poly_target_test)))
print(lasso.best_params_)

NameError: name 'GridSearchCV' is not defined

In [20]:
print("Features used: {}".format(len(list(le_poly_features_train.columns[(lasso.best_estimator_.coef_ != 0).ravel()]))))
print("Features unused: {}".format(len(list(le_poly_features_train.columns[(lasso.best_estimator_.coef_ == 0).ravel()]))))

AttributeError: 'Lasso' object has no attribute 'best_estimator_'

In [ ]:
y_pred_y = pd.DataFrame()

y_pred_y["y"] = le_poly_target_test
y_pred_y["y_pred"] = lasso.predict(le_poly_features_test)

y_pred_y

In [ ]:
y_pred_y["squared_residuals"] = (y_pred_y["y"] - y_pred_y["y_pred"])**2

y_pred_y

In [ ]:
y_pred_y["mean_target_test"] = np.repeat(np.mean(y_pred_y["y"]), y_pred_y.shape[0])

y_pred_y

In [ ]:
y_pred_y["total_squared_residuals"] = (y_pred_y["y"] - y_pred_y["mean_target_test"])**2

y_pred_y

In [ ]:
1 - (sum(y_pred_y["squared_residuals"]) / sum(y_pred_y["total_squared_residuals"]))

y_pred_y